# Model Configuration Testing

This notebook is designed for systematic hyperparameter optimization. It allows testing different combinations of model parameters to find the optimal configuration for bird song classification.

## Configuration Parameters:
- **ADAM Optimizer**: Whether to use Adam optimizer (vs SGD)
- **Early Stopping Threshold**: Patience for early stopping
- **Batch Size**: Training batch size
- **Class Weights**: Whether to use class weights for imbalanced data
- **L2 Regularization**: Weight decay parameter
- **Learning Rate Schedule**: Type and parameters for LR scheduling
- **Initial Learning Rate**: Starting learning rate
- **Standardization**: Whether to standardize features
- **SpecAugment**: Whether to apply spectrogram augmentation
- **Noise Augment**: Whether to apply Gaussian noise augmentation

In [ ]:
import os, sys
import pandas as pd
import numpy as np
import torch
from torch.utils.data import TensorDataset
import matplotlib.pyplot as plt
from datetime import datetime
import json
from typing import Dict, List, Any
import seaborn as sns
from itertools import combinations

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

# Import the clean training API
from utils.training_core import single_fold_training
from utils.models import BirdCNN
from utils.evaluation_utils import plot_single_fold_curve, print_single_fold_results

print(f"Using device: {torch.device('cuda' if torch.cuda.is_available() else 'cpu')}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name()}")

In [ ]:
# Load training data
df = pd.read_csv(os.path.join('..', 'database', 'meta', 'final', 'train_data.csv'))

print(f"Dataset shape: {df.shape}")
print(f"Number of classes: {df['label'].nunique()}")
print(f"Number of authors: {df['author'].nunique()}")

# Extract labels, authors, and features
labels = df['label'].values.astype(np.int64)
authors = df['author'].values
features = df.drop(columns=['label', 'author']).values.astype(np.float32)

# Convert to 0-1 range and reshape for CNN
features /= 255.0
features = features.reshape(-1, 1, 313, 224)

print("Features shape:", features.shape)
print("Labels shape:", labels.shape)
print("Authors shape:", authors.shape)
print("Unique classes:", len(np.unique(labels)))
print("Unique authors:", len(np.unique(authors)))

# Display class distribution
plt.figure(figsize=(12, 6))
unique_labels, counts = np.unique(labels, return_counts=True)
plt.bar(unique_labels, counts, alpha=0.7)
plt.xlabel('Class ID')
plt.ylabel('Number of Samples')
plt.title('Class Distribution in Training Data')
plt.xticks(unique_labels)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Average samples per class: {len(labels) / len(unique_labels):.1f}")

## Configuration Templates

20 different hyperparameter configurations designed for audio classification with ~3200 samples and 30 classes.

In [ ]:
# Define 20 configuration templates for systematic testing
configurations = {
    # Baseline configurations
    'config0': {
        'name': 'Conservative Baseline',
        'use_adam': True,
        'estop_thresh': 35,
        'batch_size': 24,
        'use_class_weights': False,
        'l2_regularization': 1e-4,
        'lr_schedule': None,
        'initial_lr': 0.001,
        'standardize': True,
        'spec_augment': False,
        'noise_augment': False,
        'num_epochs': 200
    },
    
    'config1': {
        'name': 'Aggressive Baseline',
        'use_adam': True,
        'estop_thresh': 25,
        'batch_size': 48,
        'use_class_weights': True,
        'l2_regularization': 5e-4,
        'lr_schedule': None,
        'initial_lr': 0.002,
        'standardize': True,
        'spec_augment': True,
        'noise_augment': True,
        'num_epochs': 250
    },
    
    # Learning rate schedule variations
    'config2': {
        'name': 'Exponential LR Decay',
        'use_adam': True,
        'estop_thresh': 40,
        'batch_size': 32,
        'use_class_weights': False,
        'l2_regularization': 1e-4,
        'lr_schedule': {'type': 'exponential', 'gamma': 0.95},
        'initial_lr': 0.003,
        'standardize': True,
        'spec_augment': True,
        'noise_augment': False,
        'num_epochs': 220
    },
    
    'config3': {
        'name': 'ReduceLROnPlateau',
        'use_adam': True,
        'estop_thresh': 50,
        'batch_size': 24,
        'use_class_weights': True,
        'l2_regularization': 2e-4,
        'lr_schedule': {'type': 'plateau', 'factor': 0.5, 'patience': 10},
        'initial_lr': 0.001,
        'standardize': True,
        'spec_augment': False,
        'noise_augment': True,
        'num_epochs': 300
    },
    
    'config4': {
        'name': 'Cosine Annealing',
        'use_adam': True,
        'estop_thresh': 30,
        'batch_size': 16,
        'use_class_weights': False,
        'l2_regularization': 1e-5,
        'lr_schedule': {'type': 'cosine', 'T_max': 50},
        'initial_lr': 0.005,
        'standardize': True,
        'spec_augment': True,
        'noise_augment': True,
        'num_epochs': 200
    },
    
    # Batch size variations
    'config5': {
        'name': 'Small Batch High LR',
        'use_adam': True,
        'estop_thresh': 35,
        'batch_size': 16,
        'use_class_weights': True,
        'l2_regularization': 3e-4,
        'lr_schedule': {'type': 'exponential', 'gamma': 0.98},
        'initial_lr': 0.004,
        'standardize': True,
        'spec_augment': True,
        'noise_augment': False,
        'num_epochs': 180
    },
    
    'config6': {
        'name': 'Large Batch Conservative',
        'use_adam': True,
        'estop_thresh': 45,
        'batch_size': 64,
        'use_class_weights': False,
        'l2_regularization': 1e-4,
        'lr_schedule': None,
        'initial_lr': 0.0005,
        'standardize': True,
        'spec_augment': False,
        'noise_augment': False,
        'num_epochs': 250
    },
    
    # Regularization focused
    'config7': {
        'name': 'Heavy Regularization',
        'use_adam': True,
        'estop_thresh': 60,
        'batch_size': 32,
        'use_class_weights': True,
        'l2_regularization': 1e-3,
        'lr_schedule': {'type': 'plateau', 'factor': 0.7, 'patience': 15},
        'initial_lr': 0.001,
        'standardize': True,
        'spec_augment': True,
        'noise_augment': True,
        'num_epochs': 300
    },
    
    'config8': {
        'name': 'Light Regularization',
        'use_adam': True,
        'estop_thresh': 25,
        'batch_size': 48,
        'use_class_weights': False,
        'l2_regularization': 1e-5,
        'lr_schedule': None,
        'initial_lr': 0.002,
        'standardize': True,
        'spec_augment': False,
        'noise_augment': False,
        'num_epochs': 150
    },
    
    # SGD variants
    'config9': {
        'name': 'SGD with Momentum',
        'use_adam': False,
        'estop_thresh': 40,
        'batch_size': 32,
        'use_class_weights': True,
        'l2_regularization': 1e-4,
        'lr_schedule': {'type': 'exponential', 'gamma': 0.9},
        'initial_lr': 0.01,
        'standardize': True,
        'spec_augment': True,
        'noise_augment': False,
        'num_epochs': 200
    },
    
    'config10': {
        'name': 'SGD High Learning Rate',
        'use_adam': False,
        'estop_thresh': 30,
        'batch_size': 24,
        'use_class_weights': False,
        'l2_regularization': 5e-4,
        'lr_schedule': {'type': 'cosine', 'T_max': 40},
        'initial_lr': 0.05,
        'standardize': True,
        'spec_augment': False,
        'noise_augment': True,
        'num_epochs': 180
    },
    
    # Augmentation focused
    'config11': {
        'name': 'Full Augmentation Suite',
        'use_adam': True,
        'estop_thresh': 45,
        'batch_size': 20,
        'use_class_weights': True,
        'l2_regularization': 2e-4,
        'lr_schedule': {'type': 'plateau', 'factor': 0.6, 'patience': 12},
        'initial_lr': 0.0015,
        'standardize': True,
        'spec_augment': True,
        'noise_augment': True,
        'num_epochs': 280
    },
    
    'config12': {
        'name': 'No Augmentation Fast',
        'use_adam': True,
        'estop_thresh': 20,
        'batch_size': 48,
        'use_class_weights': False,
        'l2_regularization': 1e-4,
        'lr_schedule': None,
        'initial_lr': 0.003,
        'standardize': False,
        'spec_augment': False,
        'noise_augment': False,
        'num_epochs': 120
    },
    
    # Class weights focus
    'config13': {
        'name': 'Balanced Classes Focus',
        'use_adam': True,
        'estop_thresh': 35,
        'batch_size': 28,
        'use_class_weights': True,
        'l2_regularization': 3e-4,
        'lr_schedule': {'type': 'exponential', 'gamma': 0.96},
        'initial_lr': 0.0012,
        'standardize': True,
        'spec_augment': True,
        'noise_augment': False,
        'num_epochs': 220
    },
    
    # Fine-tuning oriented
    'config14': {
        'name': 'Fine-tuning Style',
        'use_adam': True,
        'estop_thresh': 15,
        'batch_size': 16,
        'use_class_weights': False,
        'l2_regularization': 1e-5,
        'lr_schedule': None,
        'initial_lr': 0.0001,
        'standardize': True,
        'spec_augment': False,
        'noise_augment': False,
        'num_epochs': 100
    },
    
    # Extreme configurations for boundary testing
    'config15': {
        'name': 'High Capacity',
        'use_adam': True,
        'estop_thresh': 25,
        'batch_size': 64,
        'use_class_weights': True,
        'l2_regularization': 1e-6,
        'lr_schedule': {'type': 'cosine', 'T_max': 60},
        'initial_lr': 0.006,
        'standardize': True,
        'spec_augment': True,
        'noise_augment': True,
        'num_epochs': 240
    },
    
    'config16': {
        'name': 'Conservative Long Train',
        'use_adam': True,
        'estop_thresh': 80,
        'batch_size': 20,
        'use_class_weights': False,
        'l2_regularization': 1e-3,
        'lr_schedule': {'type': 'plateau', 'factor': 0.8, 'patience': 20},
        'initial_lr': 0.0008,
        'standardize': True,
        'spec_augment': False,
        'noise_augment': True,
        'num_epochs': 400
    },
    
    # Mixed strategies
    'config17': {
        'name': 'Adaptive Mixed',
        'use_adam': True,
        'estop_thresh': 35,
        'batch_size': 36,
        'use_class_weights': True,
        'l2_regularization': 4e-4,
        'lr_schedule': {'type': 'exponential', 'gamma': 0.94},
        'initial_lr': 0.0018,
        'standardize': True,
        'spec_augment': True,
        'noise_augment': False,
        'num_epochs': 200
    },
    
    'config18': {
        'name': 'Quick Convergence',
        'use_adam': True,
        'estop_thresh': 15,
        'batch_size': 52,
        'use_class_weights': False,
        'l2_regularization': 2e-4,
        'lr_schedule': None,
        'initial_lr': 0.0025,
        'standardize': True,
        'spec_augment': False,
        'noise_augment': False,
        'num_epochs': 100
    },
    
    'config19': {
        'name': 'Robust Generalization',
        'use_adam': True,
        'estop_thresh': 50,
        'batch_size': 24,
        'use_class_weights': True,
        'l2_regularization': 6e-4,
        'lr_schedule': {'type': 'cosine', 'T_max': 80},
        'initial_lr': 0.0014,
        'standardize': True,
        'spec_augment': True,
        'noise_augment': True,
        'num_epochs': 320
    }
}

print(f"Defined {len(configurations)} configurations for testing")
print("\nConfiguration Overview:")
for config_id, config in configurations.items():
    print(f"{config_id}: {config['name']}")

## Training Loop

Execute training for each configuration and collect results.

In [ ]:
# Initialize results storage
results_database = {}
training_start_time = datetime.now()

print("="*80)
print("STARTING CONFIGURATION TESTING")
print("="*80)
print(f"Start time: {training_start_time.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Total configurations to test: {len(configurations)}")

# Track overall progress
successful_configs = 0
failed_configs = []

for config_id, config in configurations.items():
    print(f"\n{'='*60}")
    print(f"TESTING {config_id.upper()}: {config['name']}")
    print(f"{'='*60}")
    
    config_start_time = datetime.now()
    
    try:
        # Convert config to training_core format
        training_config = {
            'num_epochs': config['num_epochs'],
            'batch_size': config['batch_size'],
            'learning_rate': config['initial_lr'],
            'use_class_weights': config['use_class_weights'],
            'early_stopping': config['estop_thresh'],
            'standardize': config['standardize'],
            'test_size': 0.2,
            'max_split_attempts': 5000,
            'min_test_segments': 5,
            'l2_regularization': config['l2_regularization'],
            'use_adam': config['use_adam'],
            'lr_schedule': config['lr_schedule']
        }
        
        # Execute training
        result = single_fold_training(
            features=features,
            labels=labels,
            authors=authors,
            model_class=BirdCNN,
            num_classes=len(np.unique(labels)),
            config=training_config,
            spec_augment=config['spec_augment'],
            gaussian_noise=config['noise_augment']
        )
        
        # Store results
        config_end_time = datetime.now()
        training_duration = (config_end_time - config_start_time).total_seconds()
        
        results_database[config_id] = {
            'config': config,
            'result': result,
            'training_time_seconds': training_duration,
            'timestamp': config_end_time.isoformat(),
            'status': 'success'
        }
        
        successful_configs += 1
        
        print(f"\n✓ {config_id} completed successfully!")
        print(f"  Final Val Accuracy: {result['final_val_acc']:.4f}")
        print(f"  Final Val F1 Score: {result['final_val_f1']:.4f}")
        print(f"  Training time: {training_duration:.1f}s ({training_duration/60:.1f}min)")
        
    except Exception as e:
        error_msg = str(e)
        print(f"\n✗ {config_id} failed: {error_msg}")
        
        failed_configs.append(config_id)
        results_database[config_id] = {
            'config': config,
            'result': None,
            'error': error_msg,
            'status': 'failed'
        }

training_end_time = datetime.now()
total_duration = (training_end_time - training_start_time).total_seconds()

print(f"\n{'='*80}")
print("CONFIGURATION TESTING COMPLETED")
print(f"{'='*80}")
print(f"End time: {training_end_time.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Total duration: {total_duration/3600:.2f} hours")
print(f"Successful configurations: {successful_configs}/{len(configurations)}")
if failed_configs:
    print(f"Failed configurations: {', '.join(failed_configs)}")

## Results Analysis

Comprehensive analysis and visualization of all configuration results.

In [ ]:
# Extract successful results for analysis
successful_results = {k: v for k, v in results_database.items() if v['status'] == 'success'}

if not successful_results:
    print("No successful configurations to analyze!")
else:
    print(f"Analyzing {len(successful_results)} successful configurations...")
    
    # Create results DataFrame
    analysis_data = []
    for config_id, data in successful_results.items():
        config = data['config']
        result = data['result']
        
        row = {
            'config_id': config_id,
            'config_name': config['name'],
            'final_val_acc': result['final_val_acc'],
            'final_val_f1': result['final_val_f1'],
            'best_val_acc': result['best_val_acc'],
            'best_val_f1': result['best_val_f1'],
            'training_time_min': data['training_time_seconds'] / 60,
            # Configuration parameters
            'use_adam': config['use_adam'],
            'estop_thresh': config['estop_thresh'],
            'batch_size': config['batch_size'],
            'use_class_weights': config['use_class_weights'],
            'l2_regularization': config['l2_regularization'],
            'has_lr_schedule': config['lr_schedule'] is not None,
            'lr_schedule_type': config['lr_schedule']['type'] if config['lr_schedule'] else 'none',
            'initial_lr': config['initial_lr'],
            'standardize': config['standardize'],
            'spec_augment': config['spec_augment'],
            'noise_augment': config['noise_augment'],
            'num_epochs': config['num_epochs']
        }
        analysis_data.append(row)
    
    results_df = pd.DataFrame(analysis_data)
    
    # Sort by F1 score (primary metric)
    results_df = results_df.sort_values('final_val_f1', ascending=False)
    
    print("TOP 10 CONFIGURATIONS BY F1 SCORE:")
    print("="*50)
    top_10 = results_df.head(10)[['config_id', 'config_name', 'final_val_f1', 'final_val_acc', 'training_time_min']]
    print(top_10.to_string(index=False))
    
    # Best configuration details
    best_config_id = results_df.iloc[0]['config_id']
    best_config_data = successful_results[best_config_id]
    
    print(f"\n🏆 BEST CONFIGURATION: {best_config_id}")
    print(f"Name: {best_config_data['config']['name']}")
    print(f"Final Val F1: {results_df.iloc[0]['final_val_f1']:.4f}")
    print(f"Final Val Accuracy: {results_df.iloc[0]['final_val_acc']:.4f}")
    print(f"Training Time: {results_df.iloc[0]['training_time_min']:.1f} minutes")

In [ ]:
# Visualization of results
if len(successful_results) > 0:
    # Create comprehensive visualization
    fig, axes = plt.subplots(2, 3, figsize=(20, 12))
    fig.suptitle('Configuration Results Analysis', fontsize=16, fontweight='bold')
    
    # 1. F1 Score comparison
    ax1 = axes[0, 0]
    results_df_plot = results_df.head(15)  # Top 15 for readability
    bars1 = ax1.bar(range(len(results_df_plot)), results_df_plot['final_val_f1'], alpha=0.7, color='skyblue')
    ax1.set_title('Final Validation F1 Score by Configuration')
    ax1.set_xlabel('Configuration Rank')
    ax1.set_ylabel('F1 Score')
    ax1.set_xticks(range(len(results_df_plot)))
    ax1.set_xticklabels(results_df_plot['config_id'], rotation=45)
    ax1.grid(axis='y', alpha=0.3)
    
    # Add value labels on bars
    for i, bar in enumerate(bars1):
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height + 0.001,
                f'{height:.3f}', ha='center', va='bottom', fontsize=8)
    
    # 2. Accuracy vs F1 scatter
    ax2 = axes[0, 1]
    scatter = ax2.scatter(results_df['final_val_acc'], results_df['final_val_f1'], 
                         c=results_df['training_time_min'], cmap='viridis', alpha=0.7, s=100)
    ax2.set_xlabel('Final Validation Accuracy')
    ax2.set_ylabel('Final Validation F1 Score')
    ax2.set_title('Accuracy vs F1 Score (colored by training time)')
    plt.colorbar(scatter, ax=ax2, label='Training Time (min)')
    
    # Add best point annotation
    best_acc = results_df.iloc[0]['final_val_acc']
    best_f1 = results_df.iloc[0]['final_val_f1']
    ax2.annotate(f'Best: {best_config_id}', xy=(best_acc, best_f1), 
                xytext=(10, 10), textcoords='offset points',
                bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.7),
                arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0'))
    
    # 3. Parameter impact - Batch size
    ax3 = axes[0, 2]
    batch_impact = results_df.groupby('batch_size')['final_val_f1'].agg(['mean', 'count']).reset_index()
    bars3 = ax3.bar(batch_impact['batch_size'], batch_impact['mean'], alpha=0.7, color='lightcoral')
    ax3.set_title('Average F1 Score by Batch Size')
    ax3.set_xlabel('Batch Size')
    ax3.set_ylabel('Average F1 Score')
    ax3.grid(axis='y', alpha=0.3)
    
    # Add count labels
    for i, (batch_size, mean_f1, count) in batch_impact.iterrows():
        ax3.text(batch_size, mean_f1 + 0.002, f'n={count}', ha='center', va='bottom', fontsize=8)
    
    # 4. Learning rate impact
    ax4 = axes[1, 0]
    lr_bins = pd.cut(results_df['initial_lr'], bins=5)
    lr_impact = results_df.groupby(lr_bins)['final_val_f1'].agg(['mean', 'count']).reset_index()
    lr_labels = [f'{interval.left:.4f}-{interval.right:.4f}' for interval in lr_impact['initial_lr']]
    bars4 = ax4.bar(range(len(lr_labels)), lr_impact['mean'], alpha=0.7, color='lightgreen')
    ax4.set_title('Average F1 Score by Learning Rate Range')
    ax4.set_xlabel('Learning Rate Range')
    ax4.set_ylabel('Average F1 Score')
    ax4.set_xticks(range(len(lr_labels)))
    ax4.set_xticklabels(lr_labels, rotation=45)
    ax4.grid(axis='y', alpha=0.3)
    
    # 5. Augmentation impact
    ax5 = axes[1, 1]
    augment_combinations = results_df.groupby(['spec_augment', 'noise_augment'])['final_val_f1'].agg(['mean', 'count']).reset_index()
    augment_labels = []
    for _, row in augment_combinations.iterrows():
        spec = 'Spec' if row['spec_augment'] else 'NoSpec'
        noise = 'Noise' if row['noise_augment'] else 'NoNoise'
        augment_labels.append(f'{Spec}+{noise}')
    
    bars5 = ax5.bar(range(len(augment_labels)), augment_combinations['mean'], alpha=0.7, color='orange')
    ax5.set_title('Average F1 Score by Augmentation Strategy')
    ax5.set_xlabel('Augmentation Combination')
    ax5.set_ylabel('Average F1 Score')
    ax5.set_xticks(range(len(augment_labels)))
    ax5.set_xticklabels(augment_labels, rotation=45)
    ax5.grid(axis='y', alpha=0.3)
    
    # Add count labels
    for i, (_, mean_f1, count) in augment_combinations.iterrows():
        ax5.text(i, mean_f1 + 0.002, f'n={count}', ha='center', va='bottom', fontsize=8)
    
    # 6. Training time vs performance
    ax6 = axes[1, 2]
    ax6.scatter(results_df['training_time_min'], results_df['final_val_f1'], alpha=0.7, s=100, color='purple')
    ax6.set_xlabel('Training Time (minutes)')
    ax6.set_ylabel('Final Validation F1 Score')
    ax6.set_title('Training Time vs Performance')
    ax6.grid(alpha=0.3)
    
    # Add trendline
    z = np.polyfit(results_df['training_time_min'], results_df['final_val_f1'], 1)
    p = np.poly1d(z)
    ax6.plot(results_df['training_time_min'], p(results_df['training_time_min']), "r--", alpha=0.8)
    
    plt.tight_layout()
    plt.show()
    
    # Save results
    results_df.to_csv('../database/meta/configuration_results.csv', index=False)
    print(f"\n💾 Results saved to ../database/meta/configuration_results.csv")

In [ ]:
# Parameter correlation analysis
if len(successful_results) > 0:
    print("\nPARAMETER CORRELATION ANALYSIS:")
    print("="*50)
    
    # Create correlation matrix for numerical parameters
    numeric_params = ['estop_thresh', 'batch_size', 'l2_regularization', 'initial_lr', 
                     'num_epochs', 'final_val_f1', 'final_val_acc', 'training_time_min']
    
    correlation_data = results_df[numeric_params].corr()
    
    # Focus on correlations with performance metrics
    f1_correlations = correlation_data['final_val_f1'].abs().sort_values(ascending=False)
    acc_correlations = correlation_data['final_val_acc'].abs().sort_values(ascending=False)
    
    print("Parameters most correlated with F1 Score:")
    for param, corr in f1_correlations.items():
        if param != 'final_val_f1':
            print(f"  {param}: {corr:.3f}")
    
    print(f"\nParameters most correlated with Accuracy:")
    for param, corr in acc_correlations.items():
        if param != 'final_val_acc':
            print(f"  {param}: {corr:.3f}")
    
    # Categorical parameter analysis
    print(f"\nCATEGORICAL PARAMETER ANALYSIS:")
    print("="*40)
    
    categorical_params = ['use_adam', 'use_class_weights', 'has_lr_schedule', 
                         'standardize', 'spec_augment', 'noise_augment']
    
    for param in categorical_params:
        if param in results_df.columns:
            grouped = results_df.groupby(param)['final_val_f1'].agg(['mean', 'std', 'count'])
            print(f"\n{param}:")
            print(grouped)
    
    # Best parameter combinations
    print(f"\nBEST PARAMETER COMBINATIONS:")
    print("="*40)
    
    # Top 3 configurations analysis
    top_3 = results_df.head(3)
    for i, (_, row) in enumerate(top_3.iterrows(), 1):
        print(f"\n#{i} - {row['config_id']} ({row['config_name']}):")
        print(f"  F1: {row['final_val_f1']:.4f}, Acc: {row['final_val_acc']:.4f}")
        print(f"  Batch Size: {row['batch_size']}, LR: {row['initial_lr']:.4f}")
        print(f"  L2: {row['l2_regularization']:.2e}, Early Stop: {row['estop_thresh']}")
        print(f"  Augmentation: Spec={row['spec_augment']}, Noise={row['noise_augment']}")
        print(f"  Optimizer: {'Adam' if row['use_adam'] else 'SGD'}, Class Weights: {row['use_class_weights']}")

## Configuration Recommendations

Based on the results, provide recommendations for future configurations.

In [ ]:
if len(successful_results) > 0:
    print("🎯 CONFIGURATION RECOMMENDATIONS")
    print("="*60)
    
    # Analyze best performing configurations
    top_5 = results_df.head(5)
    
    # Extract common patterns from top performers
    common_patterns = {}
    
    # Optimizer preference
    adam_count = top_5['use_adam'].sum()
    common_patterns['optimizer'] = 'Adam' if adam_count >= 3 else 'Mixed'
    
    # Batch size trends
    avg_batch_size = top_5['batch_size'].mean()
    common_patterns['batch_size_range'] = f"{top_5['batch_size'].min()}-{top_5['batch_size'].max()}"
    
    # Learning rate trends
    avg_lr = top_5['initial_lr'].mean()
    common_patterns['lr_range'] = f"{top_5['initial_lr'].min():.4f}-{top_5['initial_lr'].max():.4f}"
    
    # Regularization trends
    avg_l2 = top_5['l2_regularization'].mean()
    common_patterns['l2_range'] = f"{top_5['l2_regularization'].min():.2e}-{top_5['l2_regularization'].max():.2e}"
    
    # Augmentation preferences
    spec_aug_count = top_5['spec_augment'].sum()
    noise_aug_count = top_5['noise_augment'].sum()
    
    print("PATTERNS FROM TOP 5 CONFIGURATIONS:")
    print("-"*40)
    print(f"• Preferred Optimizer: {common_patterns['optimizer']}")
    print(f"• Effective Batch Size Range: {common_patterns['batch_size_range']}")
    print(f"• Optimal Learning Rate Range: {common_patterns['lr_range']}")
    print(f"• L2 Regularization Range: {common_patterns['l2_range']}")
    print(f"• SpecAugment Usage: {spec_aug_count}/5 top configs")
    print(f"• Noise Augmentation Usage: {noise_aug_count}/5 top configs")
    
    # Specific recommendations
    print(f"\nRECOMMENDED CONFIGURATION FOR FINE-TUNING:")
    print("-"*50)
    
    best_config = successful_results[results_df.iloc[0]['config_id']]['config']
    
    recommended_config = {
        'name': 'Optimized Based on Results',
        'use_adam': True,
        'estop_thresh': int(top_5['estop_thresh'].median()),
        'batch_size': int(top_5['batch_size'].median()),
        'use_class_weights': top_5['use_class_weights'].mode()[0],
        'l2_regularization': top_5['l2_regularization'].median(),
        'lr_schedule': best_config['lr_schedule'],
        'initial_lr': top_5['initial_lr'].median(),
        'standardize': True,  # Almost always beneficial
        'spec_augment': spec_aug_count >= 3,
        'noise_augment': noise_aug_count >= 3,
        'num_epochs': int(top_5['num_epochs'].median())
    }
    
    print("```python")
    print("recommended_config = {")
    for key, value in recommended_config.items():
        if isinstance(value, str):
            print(f"    '{key}': '{value}',")
        else:
            print(f"    '{key}': {value},")
    print("}")
    print("```")
    
    # Performance expectations
    expected_f1 = top_5['final_val_f1'].mean()
    f1_std = top_5['final_val_f1'].std()
    
    print(f"\nEXPECTED PERFORMANCE:")
    print(f"• F1 Score: {expected_f1:.4f} ± {f1_std:.4f}")
    print(f"• Accuracy: {top_5['final_val_acc'].mean():.4f} ± {top_5['final_val_acc'].std():.4f}")
    print(f"• Training Time: ~{top_5['training_time_min'].mean():.1f} minutes")
    
    # Areas for further exploration
    print(f"\nAREAS FOR FURTHER EXPLORATION:")
    print("-"*40)
    print("• Learning rate scheduling showed mixed results - try more variants")
    print("• Batch size optimization could be further refined")
    print("• L2 regularization sweet spot appears to be around 2e-4 to 4e-4")
    print("• Early stopping threshold could be dataset-dependent")
    print("• Consider ensemble methods combining top configurations")
    
else:
    print("❌ No successful configurations to analyze!")
    print("Check the failed configurations and adjust parameters.")

In [ ]:
# Save complete results for future reference
if len(successful_results) > 0:
    # Create a comprehensive results file
    complete_results = {
        'metadata': {
            'test_date': training_start_time.isoformat(),
            'total_configs_tested': len(configurations),
            'successful_configs': len(successful_results),
            'failed_configs': len(failed_configs),
            'total_duration_hours': total_duration / 3600,
            'dataset_info': {
                'total_samples': len(features),
                'num_classes': len(np.unique(labels)),
                'num_authors': len(np.unique(authors)),
                'feature_shape': list(features.shape)
            }
        },
        'configurations': configurations,
        'results': results_database,
        'analysis': {
            'top_10_configs': results_df.head(10).to_dict('records'),
            'parameter_correlations': correlation_data.to_dict() if 'correlation_data' in locals() else None
        }
    }
    
    # Save as JSON for future analysis
    with open('../database/meta/complete_configuration_results.json', 'w') as f:
        # Convert numpy types to native Python types for JSON serialization
        def convert_numpy(obj):
            if isinstance(obj, np.integer):
                return int(obj)
            elif isinstance(obj, np.floating):
                return float(obj)
            elif isinstance(obj, np.ndarray):
                return obj.tolist()
            return obj
        
        # Clean the data for JSON serialization
        clean_results = json.loads(json.dumps(complete_results, default=convert_numpy))
        json.dump(clean_results, f, indent=2)
    
    print("💾 Complete results saved to:")
    print("  - ../database/meta/configuration_results.csv (tabular data)")
    print("  - ../database/meta/complete_configuration_results.json (full results)")
    
    print(f"\n🎉 Configuration testing completed successfully!")
    print(f"Best configuration: {results_df.iloc[0]['config_id']} with F1 score of {results_df.iloc[0]['final_val_f1']:.4f}")
else:
    print("❌ No results to save - all configurations failed!")